In [1]:
import pandas as pd
import geopandas as gpd


# Directories
data_dir = '../data/'
landing_dir = data_dir + 'landing/'
# raw_dir = data_dir + 'raw/'

foi_sf = gpd.read_file(f"{landing_dir}FOI/GEOMARK_POLYGON.shp")

In [7]:
foi_sf

,UFI,PFI,FEATURE_ID,PARENTFTID,FTYPE,FEATSUBTYP,FEATSTATUS,NAME,NAME_LABEL,PARENTNAME,...,VICNMSTATC,THEME1,THEME2,STATE,CRDATE_PFI,SUPER_PFI,CRDATE_UFI,FEATURE_UF,FEATURE_CR,geometry
0,36114973,624178,624178,NaN,undefined,pondage_sewerage,None,SEWERAGE TREATMENT PONDS,Sewerage Treatment Ponds,None,...,11,None,None,VIC,2009-05-22,NaN,2010-03-30,36114973,2010-03-30,"MULTIPOLYGON (((146.69453 -36.54429, 146.69407..."
1,70558918,1308411,1308411,NaN,power facility,solar power station,None,STANHOPE SOLAR PROJECT 5 PTY. LTD.,Stanhope Solar Project 5 Pty. Ltd.,None,...,11,None,None,VIC,2022-08-31,NaN,2024-04-25,70558918,2024-04-25,"MULTIPOLYGON (((144.96274 -36.42865, 144.96275..."
2,39058189,837805,837805,NaN,excavation site,mine,None,CRESWICK MINE,Creswick Mine,None,...,11,None,None,VIC,2010-11-12,NaN,2010-11-12,39058189,2010-11-12,"MULTIPOLYGON (((143.88251 -37.41542, 143.88207..."
3,43644162,898053,898053,NaN,undefined,watercourse_area_channel,None,NO 20 6 CHANNEL,No 20 6 Channel,None,...,11,None,None,VIC,2011-07-12,NaN,2012-04-25,43644162,2012-04-25,"MULTIPOLYGON (((145.17781 -36.34323, 145.17781..."
4,43644298,613493,613493,NaN,undefined,watercourse_area_river,None,BELLS CREEK,Bells Creek,None,...,11,None,None,NSW,2009-05-22,NaN,2012-04-25,43644298,2012-04-25,"MULTIPOLYGON (((147.82842 -35.73061, 147.82818..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58696,71229122,650581,650581,NaN,reserve,park,None,CROXFORD PARK,Croxford Park,None,...,10,None,None,VIC,2009-12-22,NaN,2024-08-22,71229122,2024-08-22,"POLYGON ((146.39905 -37.95622, 146.39905 -37.9..."
58697,71229123,764426,764426,NaN,reserve,park,None,HADFIELD PARK,Hadfield Park,None,...,11,None,None,VIC,2010-08-27,NaN,2024-08-22,71229123,2024-08-22,"POLYGON ((144.98029 -37.41329, 144.98086 -37.4..."
58698,71230248,1280816,1280816,984975.0,sport facility,velodrome,None,None,None,CASEY FIELDS,...,11,None,None,VIC,2022-02-17,NaN,2024-08-27,71230248,2024-08-27,"POLYGON ((145.31525 -38.12828, 145.31516 -38.1..."
58699,71230249,984975,984975,NaN,sport facility,sports complex,None,CASEY FIELDS,Casey Fields,None,...,11,None,None,VIC,2012-08-29,NaN,2024-08-27,71230249,2024-08-27,"POLYGON ((145.30758 -38.11718, 145.30763 -38.1..."


In [25]:
foi_sf['geometry'] = foi_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
edited_foi_sf = foi_sf[foi_sf['STATE'] == "VIC"]
edited_foi_sf = edited_foi_sf[edited_foi_sf["FTYPE"] != "undefined"]

edited_foi_sf

,UFI,PFI,FEATURE_ID,PARENTFTID,FTYPE,FEATSUBTYP,FEATSTATUS,NAME,NAME_LABEL,PARENTNAME,CHILDEXIST,AUTHORGC,AUTHORGID,AUTHORGVER,VMADD_PFI,VICNAMESID,VICNMSTATC,THEME1,THEME2,STATE,CRDATE_PFI,SUPER_PFI,CRDATE_UFI,FEATURE_UF,FEATURE_CR,geometry
1,70558918,1308411,1308411,NaN,power facility,solar power station,None,STANHOPE SOLAR PROJECT 5 PTY. LTD.,Stanhope Solar Project 5 Pty. Ltd.,None,Y,None,None,0000/00/00,None,-2038586,11,None,None,VIC,2022-08-31,NaN,2024-04-25,70558918,2024-04-25,"MULTIPOLYGON (((144.96274 -36.42865, 144.96275..."
2,39058189,837805,837805,NaN,excavation site,mine,None,CRESWICK MINE,Creswick Mine,None,None,None,None,0000/00/00,None,-1046894,11,None,None,VIC,2010-11-12,NaN,2010-11-12,39058189,2010-11-12,"MULTIPOLYGON (((143.88251 -37.41542, 143.88207..."
6,45747370,995387,995387,NaN,storage facility,grain bunker,None,None,None,None,None,None,None,0000/00/00,54458686,-1379600,11,None,None,VIC,2013-10-02,NaN,2013-10-03,45747370,2013-10-03,"POLYGON ((145.98933 -36.03306, 145.98862 -36.0..."
8,70532277,69732,69732,1209466.0,sport facility,sports ground,None,None,None,None,None,None,None,0000/00/00,53279750,-1824473,11,None,None,VIC,2009-05-20,NaN,2024-04-23,70532277,2024-04-23,"POLYGON ((146.45795 -36.0515, 146.4578 -36.051..."
9,71123265,1247760,1247760,NaN,landform,peninsula,None,YANAKIE ISTHMUS,Yanakie Isthmus,None,Y,None,None,0000/00/00,None,-1909432,11,None,None,VIC,2020-03-23,NaN,2024-07-26,71123265,2024-07-26,"POLYGON ((146.29312 -38.88008, 146.293 -38.880..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58696,71229122,650581,650581,NaN,reserve,park,None,CROXFORD PARK,Croxford Park,None,Y,None,RES\LP144548,0000/00/00,54485059,6623,10,None,None,VIC,2009-12-22,NaN,2024-08-22,71229122,2024-08-22,"POLYGON ((146.39905 -37.95622, 146.39905 -37.9..."
58697,71229123,764426,764426,NaN,reserve,park,None,HADFIELD PARK,Hadfield Park,None,Y,None,None,0000/00/00,None,-892793,11,None,None,VIC,2010-08-27,NaN,2024-08-22,71229123,2024-08-22,"POLYGON ((144.98029 -37.41329, 144.98086 -37.4..."
58698,71230248,1280816,1280816,984975.0,sport facility,velodrome,None,None,None,CASEY FIELDS,None,None,None,0000/00/00,None,-1981730,11,None,None,VIC,2022-02-17,NaN,2024-08-27,71230248,2024-08-27,"POLYGON ((145.31525 -38.12828, 145.31516 -38.1..."
58699,71230249,984975,984975,NaN,sport facility,sports complex,None,CASEY FIELDS,Casey Fields,None,Y,None,None,0000/00/00,None,-1348884,11,None,None,VIC,2012-08-29,NaN,2024-08-27,71230249,2024-08-27,"POLYGON ((145.30758 -38.11718, 145.30763 -38.1..."


In [13]:
print(len(foi_sf))
print(len(edited_foi_sf))

58701
53289


In [20]:
pd.set_option('display.max_columns', None)  # To display all columns

edited_foi_sf

,UFI,PFI,FEATURE_ID,PARENTFTID,FTYPE,FEATSUBTYP,FEATSTATUS,NAME,NAME_LABEL,PARENTNAME,CHILDEXIST,AUTHORGC,AUTHORGID,AUTHORGVER,VMADD_PFI,VICNAMESID,VICNMSTATC,THEME1,THEME2,STATE,CRDATE_PFI,SUPER_PFI,CRDATE_UFI,FEATURE_UF,FEATURE_CR,geometry
0,36114973,624178,624178,NaN,undefined,pondage_sewerage,None,SEWERAGE TREATMENT PONDS,Sewerage Treatment Ponds,None,None,None,None,0000/00/00,None,-607364,11,None,None,VIC,2009-05-22,NaN,2010-03-30,36114973,2010-03-30,"MULTIPOLYGON (((146.69453 -36.54429, 146.69407..."
1,70558918,1308411,1308411,NaN,power facility,solar power station,None,STANHOPE SOLAR PROJECT 5 PTY. LTD.,Stanhope Solar Project 5 Pty. Ltd.,None,Y,None,None,0000/00/00,None,-2038586,11,None,None,VIC,2022-08-31,NaN,2024-04-25,70558918,2024-04-25,"MULTIPOLYGON (((144.96274 -36.42865, 144.96275..."
2,39058189,837805,837805,NaN,excavation site,mine,None,CRESWICK MINE,Creswick Mine,None,None,None,None,0000/00/00,None,-1046894,11,None,None,VIC,2010-11-12,NaN,2010-11-12,39058189,2010-11-12,"MULTIPOLYGON (((143.88251 -37.41542, 143.88207..."
3,43644162,898053,898053,NaN,undefined,watercourse_area_channel,None,NO 20 6 CHANNEL,No 20 6 Channel,None,None,None,None,0000/00/00,None,-1169164,11,None,None,VIC,2011-07-12,NaN,2012-04-25,43644162,2012-04-25,"MULTIPOLYGON (((145.17781 -36.34323, 145.17781..."
5,44789769,862925,862925,NaN,undefined,wb_lake,None,BRIMBURY SWAMP,Brimbury Swamp,None,None,None,None,0000/00/00,None,-1097396,11,None,None,VIC,2011-05-16,NaN,2013-01-25,44789769,2013-01-25,"MULTIPOLYGON (((141.21844 -37.03623, 141.21845..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58696,71229122,650581,650581,NaN,reserve,park,None,CROXFORD PARK,Croxford Park,None,Y,None,RES\LP144548,0000/00/00,54485059,6623,10,None,None,VIC,2009-12-22,NaN,2024-08-22,71229122,2024-08-22,"POLYGON ((146.39905 -37.95622, 146.39905 -37.9..."
58697,71229123,764426,764426,NaN,reserve,park,None,HADFIELD PARK,Hadfield Park,None,Y,None,None,0000/00/00,None,-892793,11,None,None,VIC,2010-08-27,NaN,2024-08-22,71229123,2024-08-22,"POLYGON ((144.98029 -37.41329, 144.98086 -37.4..."
58698,71230248,1280816,1280816,984975.0,sport facility,velodrome,None,None,None,CASEY FIELDS,None,None,None,0000/00/00,None,-1981730,11,None,None,VIC,2022-02-17,NaN,2024-08-27,71230248,2024-08-27,"POLYGON ((145.31525 -38.12828, 145.31516 -38.1..."
58699,71230249,984975,984975,NaN,sport facility,sports complex,None,CASEY FIELDS,Casey Fields,None,Y,None,None,0000/00/00,None,-1348884,11,None,None,VIC,2012-08-29,NaN,2024-08-27,71230249,2024-08-27,"POLYGON ((145.30758 -38.11718, 145.30763 -38.1..."
